## NN Structure

output policy will have a dimension of the sum of discard choices, play choices, and clue choices. clue choices has a cardinality of product of num players, num colors, and num numbers. In this case it will be 4+4+3*3*1= 17 element array

Input layer will have a dimension of the sum of opp_hand, board, discard, self_clue, and opp_clue.
opp_hand will be dummy encoding of the number of cards in the deck. board is the number of playable cards. self_clue is product of num of cards_in_hand, num of colors, and num of value, so 4*3*3 = 36. Same with opp_clue

Input layer = 15+9+15+36+36 = 111 node
also add in num bombs, number of cards left, number of clues, 4, 15-4*2=7, 8

total input layer = 111+4+7+8 = 130


In [1]:
import random
import numpy as np
from memory import Memory
from Game import *
from Player import *
import pickle
from Model import *
import config

def selfplay(model, EPISODES, memory, cards):
    scores = []
    colors = ["R","B","Y",'G',"W"]
    numbers = [1,1,1,2,2,3,3,4,4,5]

    cards = [Card(value, color) for color in colors for value in numbers]
    deck = Deck(cards = cards)

    for e in range(EPISODES):
        print(e)
        deck = Deck(cards=cards)
        p1 = Player("P1", deck.copy(), model)
        p2 = Player("P2", deck.copy(), model)

        board = Board(colors, deck.copy())
        deck.shuffle()
        p1.draw(deck.deal(4))
        p2.draw(deck.deal(4))

        players = [p1, p2]
        turn = 0
        curr = players[(turn) % 2]
        opp = players[(turn + 1) % 2]
        currGS = GS(board, deck, curr, [opp], ["Start"], turn)
        done = 0

        while done == 0:
            curr = players[(turn) % 2]
            opp = players[(turn + 1) % 2]
            turn += 1
            curr.see_players([opp])
            curr.see_board(board)
            opp.see_players([curr])
            opp.see_board(board)

            output = p1.act(currGS, tau=1)
            currGS, value, done = currGS.takeAction(output[0])
            memory.commit_stmemory(currGS, output[1])

            if done == 1:
                if memory != None:
                    for move in memory.stmemory:
                        move["value"] = value

                memory.commit_ltmemory()

        scores.append(value)
    return (scores)


memory = Memory(config.MEMORY_SIZE)
colors = ["R", "B", "Y", 'G', "W"]
numbers = [1, 1, 1, 2, 2, 3, 3, 4, 4, 5]

cards = [Card(value, color) for color in colors for value in numbers]
deck = Deck(cards=cards)

best = nn_model()
try:
    best_weights = pickle.load(open("best_weights.p", "rb" ))
    best.model.set_weights(best_weights)
    print("importing prior model")
except:
    print("No prior model")
    
scores_best = selfplay(best.model, 10, memory, cards)

C:\Users\yidin\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


importing prior model
0


TypeError: __init__() takes 2 positional arguments but 3 were given

In [2]:
new = nn_model()
new.model.set_weights(best.model.get_weights())

for i in range(config.TRAINING_LOOPS):
    minibatch = random.sample(memory.ltmemory, min(config.BATCH_SIZE, len(memory.ltmemory)))

    training_states = np.array([row['state'].nn_input for row in minibatch])
    training_targets = {'value_head': np.array([row['value'] for row in minibatch])
        , 'policy_head': np.array([row['AV'] for row in minibatch])}

    fit = new.model.fit(training_states, [training_targets["policy_head"], training_targets["value_head"]],
                        epochs=config.EPOCHS, verbose=1, validation_split=0, batch_size=32)

scores_new = selfplay(new.model, 10, memory, cards)

if sum(scores_new) > sum(scores_best):
    best_weights = new.model.get_weights()
    pickle.dump(best_weights, open("best_weights.p", "wb" ) )
    best.model.set_weights(best_weights)
    print("better model")
else:
    print("not better")

Epoch 1/1
501/501 [==============================] - 0s 686us/step - loss: 0.6869 - policy_head_loss: 0.3976 - value_head_loss: 0.9762
Epoch 1/1
501/501 [==============================] - 0s 155us/step - loss: 0.6859 - policy_head_loss: 0.3956 - value_head_loss: 0.9762
Epoch 1/1
501/501 [==============================] - 0s 185us/step - loss: 0.6863 - policy_head_loss: 0.3964 - value_head_loss: 0.9762
Epoch 1/1
501/501 [==============================] - 0s 177us/step - loss: 0.6855 - policy_head_loss: 0.3948 - value_head_loss: 0.9762
Epoch 1/1
501/501 [==============================] - 0s 149us/step - loss: 0.6850 - policy_head_loss: 0.3937 - value_head_loss: 0.9762
Epoch 1/1
501/501 [==============================] - 0s 133us/step - loss: 0.6851 - policy_head_loss: 0.3939 - value_head_loss: 0.9762
Epoch 1/1
501/501 [==============================] - 0s 174us/step - loss: 0.6850 - policy_head_loss: 0.3937 - value_head_loss: 0.9762
Epoch 1/1
501/501 [==============================] - 0s

In [3]:
scores_best

[0, 0.04, 0, 0, 0, 0.04, 0, 0, 0.04, 0]

In [4]:
scores_new

[0.04, 0, 0, 0, 0, 0, 0, 0.04, 0, 0.04]

In [5]:
len(list(memory.ltmemory))

1003

In [6]:
a = (memory.ltmemory)[-2]
a

{'AV': array([0.02040816, 0.        , 0.02040816, 0.        , 0.02040816,
        0.        , 0.        , 0.        , 0.12244898, 0.        ,
        0.08163265, 0.12244898, 0.36734694, 0.24489796]),
 'state': <Game.GS at 0x185e58bf860>,
 'value': 0.04}

In [7]:
a['state'].log

['Start',
 '1: Action P1 discarded: 3R',
 '2: Action P2 discarded: 1Y',
 '3: Action P1 discarded: 3G',
 '4: Action P2 discarded: 4R',
 '5: Action P1 discarded: 4B',
 '6: Action P2 discarded: 2G',
 '7: Action P1 discarded: 4Y',
 '8: Action P2 discarded: 3W',
 '9: Action P1 discarded: 1W',
 '10: Action P2 discarded: 4G',
 '11: Action P1 discarded: 5W',
 '12: Action P2 discarded: 3R',
 '13: Action P1 discarded: 4W',
 '14: Action P2 discarded: 1R',
 '15: Action P1 discarded: 2B',
 '16: Action P2 discarded: 3B',
 '17: Action P1 discarded: 3B',
 '18: Action P2 discarded: 2W',
 '19: Action P1 discarded: 5Y',
 '20: Action P2 discarded: 4G',
 '21: Action P1 discarded: 2Y',
 '22: Action P2 discarded: 4Y',
 '23: Action P1 discarded: 1B',
 '24: Action P2 discarded: 5B',
 '25: Action P1 discarded: 2Y',
 '26: Action P2 discarded: 1R',
 '27: Action P1 discarded: 1R',
 '28: Action P2 discarded: 1Y',
 '29: Action P1 discarded: 2R',
 '30: Action P2 discarded: 2R',
 '31: Action P1 discarded: 3G',
 '32: A

In [ ]:
best_weights = new.model.get_weights()

In [ ]:
pickle.dump(list(memory.ltmemory).copy(), open("besdfgs.p", "wb" ) )

In [ ]:
list(memory.ltmemory).copy()